In [1]:
!pip install transformers
!pip install nlp
!pip install tensorflow-text
import nlp
import transformers

In [2]:
# View all the available datasets 
print([dataset.id for dataset in nlp.list_datasets()])

['aeslc', 'ai2_arc', 'anli', 'art', 'billsum', 'blimp', 'blog_authorship_corpus', 'boolq', 'break_data', 'cfq', 'civil_comments', 'cmrc2018', 'cnn_dailymail', 'coarse_discourse', 'com_qa', 'commonsense_qa', 'coqa', 'cornell_movie_dialog', 'cos_e', 'cosmos_qa', 'crime_and_punish', 'csv', 'definite_pronoun_resolution', 'discofuse', 'drop', 'empathetic_dialogues', 'eraser_multi_rc', 'esnli', 'event2Mind', 'flores', 'fquad', 'gap', 'germeval_14', 'gigaword', 'glue', 'hansards', 'hellaswag', 'imdb', 'jeopardy', 'json', 'kor_nli', 'lc_quad', 'librispeech_lm', 'lm1b', 'math_dataset', 'math_qa', 'mlqa', 'movie_rationales', 'multi_news', 'multi_nli', 'multi_nli_mismatch', 'newsroom', 'openbookqa', 'opinosis', 'para_crawl', 'qa4mre', 'qangaroo', 'qasc', 'quarel', 'quartz', 'quoref', 'race', 'reclor', 'reddit', 'reddit_tifu', 'scan', 'scicite', 'scientific_papers', 'scifact', 'sciq', 'scitail', 'sentiment140', 'snli', 'social_i_qa', 'squad', 'squad_it', 'squad_v1_pt', 'squad_v2', 'super_glue', 't

In [3]:
# Load dataset
news_dataset = nlp.load_dataset('civil_comments')

Using custom data configuration default


In [4]:
!ls /root/.cache/huggingface/datasets/civil_comments/default/0.9.0

cache-5f029836499c4fed34cb691fa10f23e6.arrow  civil_comments-validation.arrow
civil_comments-test.arrow		      dataset_info.json
civil_comments-train.arrow		      LICENSE


In [5]:
print('training set contains: ' + str(len(news_dataset['train'])) + ' examples')
print('test set contains: ' + str(len(news_dataset['test'])) + ' examples')

training set contains: 1804874 examples
test set contains: 97320 examples


In [6]:
print('Positive example.')
print(news_dataset['train'][0])

Positive example.
{'text': "This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!", 'toxicity': 0.0, 'severe_toxicity': 0.0, 'obscene': 0.0, 'threat': 0.0, 'insult': 0.0, 'identity_attack': 0.0, 'sexual_explicit': 0.0}


In [7]:
print('Negative example.')
print(news_dataset['train'][4])

Negative example.
{'text': 'haha you guys are a bunch of losers.', 'toxicity': 0.8936170339584351, 'severe_toxicity': 0.021276595070958138, 'obscene': 0.0, 'threat': 0.0, 'insult': 0.8723404407501221, 'identity_attack': 0.021276595070958138, 'sexual_explicit': 0.0}


In [8]:
type(news_dataset['train'])

nlp.arrow_dataset.Dataset

In [9]:
!pip install tensorflow-io

In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_io.arrow as arrow_io

In [11]:
tf.__version__

'2.2.0'

In [0]:
tokenizer = transformers.BertTokenizerFast.from_pretrained('bert-base-cased')

In [0]:
def tokenize(batch_example):
  encodings = tokenizer.batch_encode_plus(batch_example['text'], pad_to_max_length=True)
  encodings.update({'toxicity': batch_example['toxicity']})
  return encodings

In [0]:
train_dataset = news_dataset['train'].map(tokenize, batched=True)

In [0]:
columns = ['input_ids', 'token_type_ids', 'attention_mask', 'toxicity']
train_dataset.set_format(type='tensorflow', columns=columns)

In [0]:
train_dataset = train_dataset[:100000]

In [0]:
features = {x: train_dataset[x] for x in columns[:3]}
labels = {'output_1': train_dataset['toxicity']}
text_train = tf.data.Dataset.from_tensor_slices((features, labels)).batch(8)

In [0]:
test_dataset = news_dataset['test'].map(tokenize, batched=True)
test_dataset.set_format(type='tensorflow', columns=columns)
test_dataset = test_dataset[:20000]
test_features = {x: test_dataset[x] for x in columns[:3]}
test_labels = {'output_1': test_dataset['toxicity']}
text_test = tf.data.Dataset.from_tensor_slices((test_features, test_labels)).batch(8)

In [0]:
from transformers import TFBertForSequenceClassification
import tensorflow_text

base_model = TFBertForSequenceClassification.from_pretrained('bert-base-cased')

In [0]:
model = tf.keras.models.Sequential([tf.keras.layers.Input(shape=[None], ragged=True, dtype='int64'), tensorflow_text.keras.layers.ToDense(), base_model])

In [0]:
loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
opt = tf.keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
to_dense (ToDense)           (None, None)              0         
_________________________________________________________________
tf_bert_for_sequence_classif ((None, 2),)              108311810 
Total params: 108,311,810
Trainable params: 108,311,810
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(text_train, validation_data=text_test, epochs=10, steps_per_epoch=3)

Epoch 1/10


KeyError: ignored